# Two Moons

In [5]:
import pickle
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from mmd import maximum_mean_discrepancy
import two_moon_sim
import config

c:\Users\u2008181\likelihood-free\sbi_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model

This notebook will demonstrate estimations of a somewhat strange Bayesian model, whose posterior evaluated at the origin $x = (0, 0)$ of the "data" will resemble two crescent moons. The forward process is a noisy non-linear transformation on a 2D plane:

$$
\begin{align*}
x_1 &= -|\theta_1 + \theta_2|/\sqrt{2} + r \cos(\alpha) + 0.25\\
x_2 &= (-\theta_1 + \theta_2)/\sqrt{2} + r\sin{\alpha}
\end{align*}
$$

with $x = (x_1, x_2)$ playing the role of "observables" (data to be learned from), $\alpha \sim \text{Uniform}(-\pi/2, \pi/2)$, and $r \sim \text{Normal}(0.1, 0.01)$ being latent variables creating noise in the data, and $\theta = (\theta_1, \theta_2)$ being the parameters that we will later seek to infer from new $x$. We set their priors to

$$
\begin{align*}
\theta_1, \theta_2 \sim \text{Uniform}(-1, 1).
\end{align*}
$$

## Methods

We consider five methods here: NPE-C, SNPE-C, NLE, SNLE, ABC-MCMC. By fixing the total number of simulation as 5000, we compare the posterior obtained by these five methods.

In [9]:
def load_post_data(method_str, simulation_budget_str, dir_prefix=""):
    d = defaultdict(dict)
    d["post_samples"] = defaultdict(dict)

    for seed in config.seeds:
        d["post_samples"][simulation_budget_str][seed] = np.loadtxt(
            f"../output/two_moons/{dir_prefix}{method_str}_post_sims{simulation_budget_str}_seed{seed}.csv",
            delimiter=",",
        )

    d = dict(d)
    d["post_samples"] = dict(d["post_samples"])

    return d

In [ ]:
# Next func: add MMD computation
# function to plot posterior samples
# function to compare MMD